In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('insurance.csv')

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [4]:
X = df.drop(columns=['expenses'])
y = df['expenses']

In [5]:
X

,age,sex,bmi,children,smoker,region
0,19,female,27.9,0,yes,southwest
1,18,male,33.8,1,no,southeast
2,28,male,33.0,3,no,southeast
3,33,male,22.7,0,no,northwest
4,32,male,28.9,0,no,northwest
...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest
1334,18,female,31.9,0,no,northeast
1335,18,female,36.9,0,no,southeast
1336,21,female,25.8,0,no,southwest


In [6]:
y

0       16884.92
1        1725.55
2        4449.46
3       21984.47
4        3866.86
          ...   
1333    10600.55
1334     2205.98
1335     1629.83
1336     2007.95
1337    29141.36
Name: expenses, Length: 1338, dtype: float64

In [7]:
numerical_columns = X.select_dtypes(exclude='object').columns
categorical_columns = X.select_dtypes(include='object').columns

In [8]:
numerical_columns

Index(['age', 'bmi', 'children'], dtype='object')

In [9]:
categorical_columns

Index(['sex', 'smoker', 'region'], dtype='object')

In [11]:
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", OneHotEncoder(), categorical_columns),
        ("StandardScaler", StandardScaler(), numerical_columns),        
    ]
)

In [12]:
X_transformed = preprocessor.fit_transform(X)

In [13]:
X_transformed.shape

(1338, 11)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed,y,test_size=0.2,random_state=42)

In [15]:
X_train.shape

(1070, 11)

In [16]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [17]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Adaboost Regressor": AdaBoostRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor()
}

In [18]:
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 6149.8267
- Mean Absolute Error: 4163.0462
- R2 Score: 0.7380
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5812.6837
- Mean Absolute Error: 4115.3098
- R2 Score: 0.7824


Lasso
Model performance for Training set
- Root Mean Squared Error: 6105.2476
- Mean Absolute Error: 4208.0337
- R2 Score: 0.7418
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5797.2951
- Mean Absolute Error: 4182.3918
- R2 Score: 0.7835


Ridge
Model performance for Training set
- Root Mean Squared Error: 6105.3089
- Mean Absolute Error: 4212.3760
- R2 Score: 0.7417
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5798.5673
- Mean Absolute Error: 4187.2631
- R2 Score: 0.7834


Decision Tree
Model performance for Training set
- Root Mean Squared Error: 494.2061
- Mean Absolute Error: 29.5725
- R2 Score:

In [19]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
6,Gradient Boosting Regressor,0.878850
4,Random Forest Regressor,0.861100
5,Adaboost Regressor,0.830095
1,Lasso,0.783518
2,Ridge,0.783422
0,Linear Regression,0.782367
3,Decision Tree,0.749644


# Hyperparameter Tuning for Gradient Boosting

In [20]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'max_depth': [2, 3, 4, 5, 6, 7, 8]
}

In [21]:
grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=10, n_jobs=-1, verbose=1)

In [22]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 210 candidates, totalling 2100 fits


GridSearchCV(cv=10, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [50, 100, 150, 200, 250, 300]},
             verbose=1)

In [23]:
grid_search.best_params_

{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}

In [24]:
final_model = GradientBoostingRegressor(n_estimators=grid_search.best_params_['n_estimators'], 
                                        max_depth=grid_search.best_params_['max_depth'],
                                        learning_rate=grid_search.best_params_['learning_rate'])

In [25]:
final_model.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05)

In [27]:
print(final_model.score(X_test, y_test))

0.8811272007394277


In [28]:
models = {
    "Gradient Boosting Regressor": final_model
}

In [29]:
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Gradient Boosting Regressor
Model performance for Training set
- Root Mean Squared Error: 4138.5048
- Mean Absolute Error: 2299.5442
- R2 Score: 0.8813
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4295.9110
- Mean Absolute Error: 2449.3113
- R2 Score: 0.8811




In [30]:
y_pred = final_model.predict(X_test)

In [32]:
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is:", score)

 Accuracy of the model is: 88.11272007394277


In [33]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df

,Actual Value,Predicted Value,Difference
764,9095.07,10496.126192,-1401.056192
887,5272.18,6106.333241,-834.153241
890,29330.98,27249.913444,2081.066556
1293,9301.89,9876.907445,-575.017445
259,33750.29,34569.869308,-819.579308
...,...,...,...
109,47055.53,45352.561740,1702.968260
575,12222.90,12339.028895,-116.128895
535,6067.13,7229.705688,-1162.575688
543,63770.43,46972.421618,16798.008382
